In [28]:
import sys
import os

In [29]:
project_root = "C:/Users/Rohith Kumar/OneDrive/Desktop/Text_moderization_project"
sys.path.append(project_root)

In [30]:
from utils.preprocessing import load_csv_folder, full_preprocessing
from models.text_model import train_bert_model, predict_texts
from sklearn.model_selection import train_test_split

In [31]:
folder_path = "C:/Users/Rohith Kumar/OneDrive/Desktop/Text_moderization_project/data/text"
df = load_csv_folder(folder_path)
df = full_preprocessing(df, text_column="comment_text")

In [32]:
folder_path = "C:/Users/Rohith Kumar/OneDrive/Desktop/Text_moderization_project/data/text"
df = load_csv_folder(folder_path)
df = full_preprocessing(df, text_column="comment_text")

label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df = df.dropna(subset=label_cols)
df = df[(df[label_cols] != -1).all(axis=1)]

x = df['lemmatized_text']
y = df[label_cols].fillna(0).astype(int)

In [33]:
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
df = df.dropna(subset=label_cols)
df = df[(df[label_cols] != -1).all(axis=1)]

In [34]:
x = df['lemmatized_text']
y = df[label_cols].fillna(0).astype(int)

In [35]:
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [50]:
x_train_small = x_train.sample(n=1500, random_state=42)
y_train_small = y_train.loc[x_train_small.index]

x_test_small = x_test.sample(n=600, random_state=42)
y_test_small = y_test.loc[x_test_small.index]


In [51]:
model, tokenizer, trainer, results = train_bert_model(
    x_train_small, y_train_small,
    
    x_test_small, y_test_small,
    epochs=3,
    batch_size=8,
    threshold=0.3  
)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users/Rohith Kumar/OneDrive/Desktop/Text_moderization_project\models\text_model.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  )
c:\Users\Rohith Kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
20,0.531000
40,0.262200
60,0.138100
80,0.108300
100,0.062600
120,0.090500
140,0.116700
160,0.124700
180,0.108400
200,0.078500


c:\Users\Rohith Kumar\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [52]:
print("\nEvaluation Results:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Evaluation Results:
eval_loss: 0.0702
eval_accuracy: 0.9819
eval_precision: 0.7059
eval_recall: 0.4186
eval_f1: 0.5255
eval_runtime: 107.9403
eval_samples_per_second: 5.5590
eval_steps_per_second: 0.6950
epoch: 3.0000


In [53]:
sample_texts = ["You are so stupid!", "I love this product!"]
preds, probs = predict_texts(model, tokenizer, sample_texts)
print("Predictions:", preds)
print("Probabilities:", probs)

Predictions: tensor([[1, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 0]], dtype=torch.int32)
Probabilities: tensor([[0.6980, 0.2084, 0.4653, 0.1295, 0.3637, 0.1270],
        [0.0528, 0.0094, 0.0196, 0.0081, 0.0168, 0.0077]])


In [2]:
import os

save_dir = "./Backend/toxic_bert_model"  # path inside your project
os.makedirs(save_dir, exist_ok=True)

# Save model
model.save_pretrained(save_dir)

# Save tokenizer
tokenizer.save_pretrained(save_dir)


('./Backend/toxic_bert_model\\tokenizer_config.json',
 './Backend/toxic_bert_model\\special_tokens_map.json',
 './Backend/toxic_bert_model\\vocab.txt',
 './Backend/toxic_bert_model\\added_tokens.json',
 './Backend/toxic_bert_model\\tokenizer.json')